In [1]:
import numpy as np
import json
import cv2
from hausdorff_dist import hausdorff_distance

In [ ]:
def check_center_inside_with_roi(center, mask):
    if np.sum(mask[center[1], center[0]]) > 100:
        return True
    else:
        return False


def check_bbox_inside_with_roi(bbox, mask):
    # check if four point of bbox all in roi area
    is_inside = True

    x_tl = bbox[1]
    y_tl = bbox[2]
    x_br = bbox[3]
    y_br = bbox[4]

    for x in [x_tl, x_br]:
        if x <= 0 or x >= mask.shape[1]:
            return False

    for y in [y_tl, y_br]:
        if y <= 0 or y >= mask.shape[0]:
            return False

    vertexs = [[x_tl, y_tl], [x_tl, y_br], [x_br, y_tl], [x_br, y_br]]
    for v in vertexs:
        (g, b, r) = mask[v[1], v[0]]
        if (g, b, r) < (128, 128, 128):
            is_inside = False
            return is_inside

    return is_inside


def check_tracks_with_roi(tracks, mask, center=True):
    tracks_end_in_roi = []
    tracks_start_in_roi = []
    tracks_too_short = []
    checker = check_center_inside_with_roi if center else check_bbox_inside_with_roi
    for trackid, track in tracks.items():
        if center:
            start_frame = track['tracklet'][0]
            end_frame = track['tracklet'][-1]
        else:
            start_frame = track['bbox'][0]
            end_frame = track['bbox'][-1]

        if checker(start_frame, mask):
            if track['startframe'] > 1:
                tracks_start_in_roi.append(trackid)

        if checker(end_frame, mask):
            tracks_end_in_roi.append(trackid)

        if track['endframe'] - track['startframe'] < 10:
            if trackid not in tracks_start_in_roi:
                tracks_too_short.append(trackid)
    return tracks_end_in_roi, tracks_start_in_roi, tracks_too_short


def check_bbox_overlap_with_roi(bbox, mask):
    is_overlap = False
    if bbox[1] >= mask.shape[1] or bbox[2] >= mask.shape[0] \
            or bbox[3] < 0 or bbox[4] < 0:
        return is_overlap

    x_tl = bbox[1] if bbox[1] > 0 else 0
    y_tl = bbox[2] if bbox[2] > 0 else 0
    x_br = bbox[3] if bbox[3] < mask.shape[1] else mask.shape[1] - 1
    y_br = bbox[4] if bbox[4] < mask.shape[0] else mask.shape[0] - 1
    vertexs = [[x_tl, y_tl], [x_tl, y_br], [x_br, y_tl], [x_br, y_br]]
    for v in vertexs:
        (g, b, r) = mask[v[1], v[0]]
        if (g, b, r) > (128, 128, 128):
            is_overlap = True
            return is_overlap

    return is_overlap


def distance_of_2_points(p1, p2):
    return np.sqrt(np.sum(np.square(np.array(p2) - np.array(p1))))


def is_same_direction_by_dist(checked_dir, base_dir):
    endpoint_checked = checked_dir[-1]
    startpoint_base = base_dir[0]
    endpoint_base = base_dir[-1]

    end_dis = distance_of_2_points(endpoint_checked, endpoint_base)
    start_dis = distance_of_2_points(endpoint_checked, startpoint_base)
    if end_dis < start_dis:
        return True
    else:
        return False


def is_same_direction(traj1, traj2, angle_thr):
    vec1 = np.array([traj1[-1][0] - traj1[0][0], traj1[-1][1] - traj1[0][1]])
    vec2 = np.array([traj2[-1][0] - traj2[0][0], traj2[-1][1] - traj2[0][1]])
    L1 = np.sqrt(vec1.dot(vec1))
    L2 = np.sqrt(vec2.dot(vec2))
    if L1 == 0 or L2 == 0:
        return False
    cos = vec1.dot(vec2) / (L1 * L2)
    angle = np.arccos(cos) * 360 / (2 * np.pi)
    if angle < angle_thr:
        return True
    else:
        return False


def calc_angle(vec1, vec2):
    vec1 = np.array([traj1[-1][0] - traj1[-5][0], traj1[-1][1] - traj1[-5][1]])
    vec2 = np.array([traj2[-1][0] - traj2[-5][0], traj2[-1][1] - traj2[-5][1]])
    L1 = np.sqrt(vec1.dot(vec1))
    L2 = np.sqrt(vec2.dot(vec2))
    if L1 == 0 or L2 == 0:
        return 90
    cos = vec1.dot(vec2) / (L1 * L2)
    if cos > 1:
        return 90
    angle = np.arccos(cos) * 360 / (2 * np.pi)
    return angle

In [ ]:
def load_tracking(track_file):
    tracks = {}
    with open(track_file, 'r') as ft:
        lines = [line.strip('\n').split(' ') for line in ft]
        for line in lines:
            frameid = int(line[1])
            trackid = int(line[2])
            x1 = max(int(float(line[3])), 0)
            y1 = max(int(float(line[4])), 0)
            x2 = min(int(float(line[5])), 1280)
            y2 = min(int(float(line[6])), 720)
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            label = line[7]
            if trackid in tracks:
                tracks[trackid]['endframe'] = frameid
                tracks[trackid]['bbox'].append([frameid, x1, y1, x2, y2, label])
                tracks[trackid]['tracklet'].append([cx, cy])
            else:
                tracks[trackid] = {'startframe': frameid,
                                   'endframe': frameid,
                                   'bbox': [[frameid, x1, y1, x2, y2, label]],
                                   'tracklet': [[cx, cy]],
                                   'video_idx': line[0]}
    return tracks


In [ ]:
tracks = load_tracking("track_results/cam_01.txt")
mask= cv2.imread("mask/cam_01.png")
h, w, c = mask.shape

In [ ]:
tracks_end_in_roi, tracks_start_in_roi, tracks_too_short = check_tracks_with_roi(tracks, mask)

In [ ]:
print("End ROI :",len(tracks_end_in_roi))
print("Start ROI :",len(tracks_start_in_roi))
print("Too short :",len(tracks_too_short))
A = set(tracks_start_in_roi)
B = set(tracks_end_in_roi)
overlap = list(A.intersection(B))
print("OVERLAP :",len(overlap))

In [ ]:
tracks

In [ ]:
1+!